<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#한국 버블 커뮤니티 오픈톡방 대화의 질-답, 모두의노코드 질-답+댓 각각을 요약해 저장하는 코드(로컬)

In [ ]:
!pip install datasets

import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
from datasets import Dataset, load_dataset
from google.colab import drive
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel, PreTrainedTokenizerFast, BartForConditionalGeneration
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
Mounted at /content/drive


In [ ]:
# Load Model and Tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
#카카오톡 질답세션 불러오고 Dataset으로 받기
file_path = '/content/drive/My Drive/judge_answer_result_KcBERT_short_questiontrained.csv'

df1 = pd.read_csv(file_path, index_col = 0)

#커뮤니티 데이터와 결합하기 위해 url 열 추가
df1['url'] = [''] * len(df1)

# dataset = load_dataset("csv", data_files=file_path, column_names=["question", "answer", "questioner", "respondent", "date", "url"])

print(df1)

                                               question  \
2      혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;   
8      API통신을 위해 임시 토큰 만료 전 재발급을 주기적으로 해야하는데, 이걸 버블 ...   
9      안녕하세요~ 프로모션 적용하려면 플랜 업그레이드를 별도로 진행해야하는걸까요 아니면...   
13     감사합니다. 버블에서도 매 n시간 단위로 특정 api 콜을 예약하는 loop를 만...   
15     혹시 워드프레스도 노코드 툴 중 하나인가요? 버블 softr 웹플로우 아임웹 글라...   
...                                                 ...   
1597   안녕하세요. 제작 중이던 서비스 이름이 변경되면서 앱 이름을 바꾸려고 하는데 아무...   
1598      혹시 버블 증상이 로딩이 오래 걸리는거 말씀하시나용 저도 지금 버블이 안켜져용ㅋㅋ   
1599   혹시 포스기 소리를 제작 혹은 외주를 맡겨보신 경험이 있는 분 이곳에 계실까요? ...   
1601   혹시 버블에서 도움말 기능을 만들고 싶은데, 추천하시는 강의가 있나요??도움말 기...   
1602   Workflow error - List at maximum length (1000...   

                                                 answer      questioner  \
2     [' 저희는 히로인스라는 서비스를 운영하고 있는데요. 백엔드만 버블을 쓰고 있습니다...        IT / 돔돔철   
8     ['  API통신을 위해 임시 토큰 만료 전 재발급을 주기적으로 해야하는데, 이걸 ...    김범준 / 기획, 개발   
9     [' 안녕하세요? 프로모션은 자동 적용됩니다! 단, 콜수나 유저수에 따라 플랜 비용...        기뻐하

In [ ]:
#모두의노코드 질-답+댓 세션 불러오고 Dataset으로 받기
file_path = '/content/drive/My Drive/community_qna_preprocessed.xlsx'

df2 = pd.read_excel(file_path, index_col = 0)

#카카오톡 데이터와 결합하기 위해 답변 작성자 열 추가
df2['respondent'] = [''] * len(df2)

#Slug가 존재하는 경우만 질문 원글로 판별
df2 = df2[pd.isna(df2['Slug']) == False]
#답변이 달리지 않은 질문글은 제외, 추후 댓글까지 활용 예정
df2 = df2[pd.isna(df2['답변']) == False]

#필요없는 열 삭제
df2 = df2.drop(['댓글', '관련 링크', '제목', '첨부 파일', 'Slug'], axis = 1)

#카카오톡 데이터와 결합하기 위해 열 이름 변경
df2 = df2.rename(columns = {'내용' : 'question', '답변' : 'answer', '작성일' : 'date', '작성자' : 'questioner', '게시글 링크' : 'url'})

# dataset = load_dataset("csv", data_files = file_path, column_names = ["question", "answer", "questioner", "respondent", "date", "url"])

print(df2)

                                              question  \
0    안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
2    안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
4    안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
9    모두의노코드 만드신 분들께 질문 드리고 싶어요.제가 버블로 만든 서비스 관련해 모바...   
11   한번에 여러개의 파일을 업로드하는 기능을 구현하기 위해서 버블 무료 플러그인 Dro...   
..                                                 ...   
542  스포츠 클럽을 만들고 클럽원들의 신청을 받아서 허가해주고, 클럽원들의 매치 기록을 ...   
545  custom state 만들어서 하고 있는데 파란색이지만  오류가 나고 있어요당연히...   
549  정렬 기능을 사용중인데아래의 그림과 같이 2가지의 문제가 해결되지 않고 있습니다 ....   
558  버블에서 아래 UI처럼 +버튼을 누르면 하위에 계속해서 폴더가 추가되도록 프론트를 ...   
575  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   

                                                answer        date questioner  \
0    저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...  2023-01-11        김정민   
2    헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...  2023-01-12        김태연   
4    충분히 버블로 개발가능할 것 같아요! 이미 비슷한 수준의 어플을 개발한 사례가 많아...  2023

In [ ]:
print(type(df1.iloc[2]['date']))

<class 'str'>


In [ ]:
# df2['date'] = df2['date'].astype(str)

In [ ]:
print(type(df2.iloc[3]['date']))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [ ]:
# #두 데이터프레임 합치고 Dataset으로 변환하기
df = pd.concat([df1, df2], ignore_index = True)
print(df)
dataset = Dataset.from_pandas(df)

print(dataset)

                                              question  \
0     혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;   
1     API통신을 위해 임시 토큰 만료 전 재발급을 주기적으로 해야하는데, 이걸 버블 ...   
2     안녕하세요~ 프로모션 적용하려면 플랜 업그레이드를 별도로 진행해야하는걸까요 아니면...   
3     감사합니다. 버블에서도 매 n시간 단위로 특정 api 콜을 예약하는 loop를 만...   
4     혹시 워드프레스도 노코드 툴 중 하나인가요? 버블 softr 웹플로우 아임웹 글라...   
..                                                 ...   
769  스포츠 클럽을 만들고 클럽원들의 신청을 받아서 허가해주고, 클럽원들의 매치 기록을 ...   
770  custom state 만들어서 하고 있는데 파란색이지만  오류가 나고 있어요당연히...   
771  정렬 기능을 사용중인데아래의 그림과 같이 2가지의 문제가 해결되지 않고 있습니다 ....   
772  버블에서 아래 UI처럼 +버튼을 누르면 하위에 계속해서 폴더가 추가되도록 프론트를 ...   
773  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   

                                                answer     questioner  \
0    [' 저희는 히로인스라는 서비스를 운영하고 있는데요. 백엔드만 버블을 쓰고 있습니다...       IT / 돔돔철   
1    ['  API통신을 위해 임시 토큰 만료 전 재발급을 주기적으로 해야하는데, 이걸 ...   김범준 / 기획, 개발   
2    [' 안녕하세요? 프로모션은 자동 적용됩니다! 단, 콜수나 유저수에 따라 플랜 비용...       기뻐하는 라이언   
3    [' Pow

In [ ]:
# Ensure the classifier is using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [ ]:
# # 데이터셋 전처리 (예제)
# texts = ["This is a long text that needs to be summarized.", "Here's another text for summarization."]
# tokenized_inputs = tokenizer(texts, padding="longest", truncation=True, return_tensors="pt", max_length=1024)

# print(tokenized_inputs)

{'input_ids': tensor([[    0, 15054,   303, 15868,  1700, 15868, 22911, 25674, 24687, 18090,
         23795,   315, 19133, 16881, 21514,   300, 17752,   314, 27141, 19306,
           300, 18482,   316, 20461, 15562, 26147, 17752,   245,     1],
        [    0, 16391, 14879,   300,   238,   314,  1700, 15195,   310, 16160,
         14879, 18090, 23795,   315, 21579, 18482,   316, 20461, 15562, 26147,
         19334,   245,     1,     3,     3,     3,     3,     3,     3]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
         0, 0, 0, 0, 0]])}


In [ ]:
# 데이터셋 질문 전처리 함수, Dataset 열을 인자로 받는다.
def preprocess_question(examples):
    return tokenizer(examples['question'], return_tensors = "pt", truncation=True, padding="max_length", max_length=1026)


In [ ]:
#데이터를 gpu로 이동시킬 때 DataLoader에서 사용되는 collate_fn 함수
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids'], dtype = torch.long) for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask'], dtype = torch.long) for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask}

In [ ]:
#모델로 question summary 리스트 생성
question_summary = []

# Encoding question
question_text_dict = {"question" : dataset["question"][1:]} #dataset["question"]의 첫 요소인 'question'삭제
question_text = Dataset.from_dict(question_text_dict)
tokenized_question_inputs = question_text.map(preprocess_question, num_proc = 4, batched = True, batch_size = 128)

#데이터를 gpu로 이동시킬 때 DataLoader에서 사용되는 collate_fn 함수
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids'], dtype = torch.long) for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask'], dtype = torch.long) for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask}

loader = DataLoader(tokenized_question_inputs, batch_size=16, collate_fn=collate_fn)

# Question summary ids 생성,배치 단위
for batch in loader:
  #배치 단위로 loader에서 gpu로 데이터 전송
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)

  question_summary_ids = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    bos_token_id = model.config.bos_token_id,
    eos_token_id = model.config.eos_token_id,
    length_penalty = 1.0,
    max_length = 40,
    min_length = 5,
    num_beams = 6,
    repetition_penalty = 1.5,
    no_repeat_ngram_size = 3,
  )
  #생성한 Question summary ids를 다시 tokenizer가 있는 cpu로 이동
  question_summary_ids = question_summary_ids.cpu()

  #배치마다 answer_summary_ids를 생성하기 때문에 cpu로 옮겨진 answer_summary_ids 배치 1개를 answer_summary 리스트에 옮기기
  question_summary.append(question_summary_ids)

# Decoding question summary (list)
qsum = []

#배치 단위로 decode 처리
for question_summary_result in question_summary:
  qsum_batch = [tokenizer.decode(question_summary_result[i], skip_special_tokens = True) for i in tqdm(range(len(question_summary_result)), desc = "Decoding question summary")]
  qsum.append(qsum_batch)

#배치 풀기
qsum = sum(qsum, [])

print(qsum[:5])


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/773 [00:00<?, ? examples/s]

Decoding question summary: 100%|██████████| 5/5 [00:00<00:00, 10506.77it/s]

['API통신을 위해 임시 토큰 만료 전 재발급을 주기적으로 해야하는데, 이걸 버블에서 자동화 할 수 있을까?', '프로모션을 적용하려면 플랜 업그레이드를 별도로 진행해야하는지, 자동 적용되는지요?', '버블에서도 n시간 단위로 특정 api 콜을 예약하는 loop를 만들 수 있다고 알고있는데, 이걸로는 어려울까요? 해보신 분 계신가요?', '워드프레스도 노코드 툴 중 하나인가요? 버블 softr 웹플로우 아임웹 글라이드는 언급되는데 워드프레스는', '버블 어렵대서 카페24로 예약기능 가능한 홈피 만드려하니 워프로 하더라고 워프나 버블 러닝커브가 비슷하다면 버블을 배우는']


In [ ]:
# 데이터셋 답변 전처리 함수, Dataset 열을 인자로 받는다. 질문 전처리 함수와 공통적으로 사용하지 않은 이유는 examples['answer']형태로 입력하지 않으면 오류가 발생한다.
def preprocess_answer(examples):
    return tokenizer(examples['answer'], return_tensors = "pt", truncation=True, padding="max_length", max_length=1026)


In [ ]:
#모델로 answer summary 리스트 생성
answer_summary = []

# Encoding answer
answer_text_dict = {"answer" : dataset["answer"][1:]} #dataset["answer"]의 첫 요소인 'answer'삭제
answer_text = Dataset.from_dict(answer_text_dict)
tokenized_answer_inputs = answer_text.map(preprocess_answer, num_proc = 4, batched = True, batch_size = 128)

loader = DataLoader(tokenized_answer_inputs, batch_size=16, collate_fn=collate_fn)

# Answer summary ids 생성,배치 단위, question때와 다른 하이퍼파라미터 필요(요약길이)
for batch in loader:
  #배치 단위로 loader에서 gpu로 데이터 전송
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)

  answer_summary_ids = model.generate(
    input_ids = input_ids,
    attention_mask = attention_mask,
    bos_token_id = model.config.bos_token_id,
    eos_token_id = model.config.eos_token_id,
    length_penalty = 1.0,
    max_length = 70,
    min_length = 5,
    num_beams = 6,
    repetition_penalty = 1.5,
    no_repeat_ngram_size = 3,
  )

  #생성한 Answer summary ids를 다시 tokenizer가 있는 cpu로 이동
  answer_summary_ids = answer_summary_ids.cpu()

  #배치마다 answer_summary_ids를 생성하기 때문에 cpu로 옮겨진 answer_summary_ids 배치 1개를 answer_summary 리스트에 옮기기
  answer_summary.append(answer_summary_ids)

# Decoding question summary (list)
asum = []

#배치 단위로 decode 처리
for answer_summary_result in answer_summary:
  asum_batch = [tokenizer.decode(answer_summary_result[i], skip_special_tokens = True) for i in tqdm(range(len(answer_summary_result)), desc = "Decoding answer summary")]
  asum.append(asum_batch)

#배치 풀기
asum = sum(asum, [])

print(asum[:5])


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/773 [00:00<?, ? examples/s]

Decoding answer summary: 100%|██████████| 5/5 [00:00<00:00, 10215.06it/s]

['API통신을 위해 임시 토큰 만료 전 재발급을 주기적으로 해야하는데, 이걸 버블에서 어떻게 자동화 할 수 있을까요?', 'Jobs는 스케쥴러에 등록할 수 있는 task라고 보시면 됩니다.', '작업을 배치할 때 최초에는 HTTP Receiver를 넣고 로직을 처리하고 마지막에 응답 작업을 배치하지 않으면 클라이언트에서 API를 콜하고 응답 여부에 관계없이 바로 다른 일을 수행한다.', '워드프레스는 에이전시나 능력자들이 템플릿을 마개조(?)해서 여러 방면에서 쓰이면서 지금은 하나의 프레임웤처럼 작동하는것 같습니다.', '워프, 카페24의 초기 러닝 커브는 버블이 높지만 나중에 자유도가 더 높아서 할 수 있는 게 많아진다. 워프는 호스팅을 설정해야하는 어려움만 극복하면 나머지는 아임웹과 유사했다.']


In [ ]:
df = pd.DataFrame(asum, columns = ['answer_summary'])
print(df)

                                        answer_summary
0    API통신을 위해 임시 토큰 만료 전 재발급을 주기적으로 해야하는데, 이걸 버블에서...
1                 Jobs는 스케쥴러에 등록할 수 있는 task라고 보시면 됩니다.
2    작업을 배치할 때 최초에는 HTTP Receiver를 넣고 로직을 처리하고 마지막에...
3    워드프레스는 에이전시나 능력자들이 템플릿을 마개조(?)해서 여러 방면에서 쓰이면서 ...
4    워프, 카페24의 초기 러닝 커브는 버블이 높지만 나중에 자유도가 더 높아서 할 수...
..                                                 ...
768          클럽이라는 데이터가 여러개 Create 되면클럽 리스트가 되는 형태랍니다.
769  박재은님의 답변주신 내용으로 채택합니다 타입이 안 맞아서 생기는 오류입니다~, 박재...
770  두어서치포 내에서만 했을 때 정렬이 안 되었는데 밖에 sorted by를 사용하여 ...
771  DB에 폴더 테이블이 있고 필드에 상위폴더라는 필드를 만들고 타입을 지금 만든 폴더...
772  버블로 외주개발하고 있는 리트머스팀입니다. 링크로 톡 남겨놓으시면 견적을 도와드리겠...

[773 rows x 1 columns]


In [ ]:
#데이터셋에 새로운 열 추가
print(len(dataset))

dataset_dict = {'question' : dataset['question'][1:], 'answer' : dataset['answer'][1:], 'question_summary' : qsum, 'answer_summary' : asum, 'questioner' : dataset['questioner'][1:], 'respondent' : dataset['respondent'][1:], 'date' : dataset['date'][1:], 'url' : dataset['url'][1:]}
dataset = Dataset.from_dict(dataset_dict)

print(dataset)
# dataset.add_column("qsum", qsum)
# dataset.add_column("asum", asum)

774
Dataset({
    features: ['question', 'answer', 'question_summary', 'answer_summary', 'questioner', 'respondent', 'date', 'url'],
    num_rows: 773
})


In [ ]:
#.csv 파일로 google drive에 저장
save_path = '/content/drive/My Drive/summary_result_short.csv'

dataset.to_csv(save_path)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1090754